# Entrenamiento y evaluación.

## Importaciones.

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest
from sklearn.svm import SVC
from sklearn.impute import SimpleImputer

from sklearn import metrics

## Normalización.

In [4]:
df = pd.read_csv("../data/raw/coches_segunda_mano_modelos.csv")

In [5]:
x = df[["year", "kms", "power", "antiquity", "num_make"]]
y = df["price"]

scaler = StandardScaler()
x_scaler = scaler.fit_transform(x)

x_train, x_test, y_train, y_test = train_test_split(x_scaler, y, test_size=0.2, random_state=10)

## Modelos.

### Modelo Lineal Regresion.

In [6]:
modelo_lr = LinearRegression()
modelo_lr.fit(x_train, y_train)

predictions = modelo_lr.predict(x_test)

print("MAE:", metrics.mean_absolute_error(y_test, predictions))
print("MAPE:", metrics.mean_absolute_percentage_error(y_test, predictions))
print("MSE:", metrics.mean_squared_error(y_test, predictions))
print("RMSE:", np.sqrt(metrics.mean_squared_error(y_test, predictions)))
print("r2_score train", modelo_lr.score(x_train, y_train))
print("r2_score test",modelo_lr.score(x_test, y_test))

MAE: 4487.438291166739
MAPE: 0.5116388945296748
MSE: 48313149.71569854
RMSE: 6950.766124370647
r2_score train 0.7268439407466206
r2_score test 0.7126410091983857


In [ ]:
import pickle

with open('../models/modelo_lr.pkl', "wb") as archivo:
    pickle.dump(modelo_lr, archivo)

### Modelo Lineal Regresion con regresion polinomial de 3 

In [8]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
x_scaler = scaler.fit_transform(x)


poly_feats = PolynomialFeatures(degree = 3)
poly_feats.fit(x_scaler)
X_poly = poly_feats.transform(x_scaler)

x_train, x_test, y_train, y_test = train_test_split(X_poly,y, test_size = 0.2, random_state=12)

lin_reg = LinearRegression()
lin_reg.fit(x_train, y_train)

predictions = lin_reg.predict(x_test)

print("MAE:", metrics.mean_absolute_error(y_test, predictions))
print("MAPE:", metrics.mean_absolute_percentage_error(y_test, predictions))
print("MSE:", metrics.mean_squared_error(y_test, predictions))
print("RMSE:", np.sqrt(metrics.mean_squared_error(y_test, predictions)))
print("r2_score train", lin_reg.score(x_train, y_train))
print("r2_score test",lin_reg.score(x_test, y_test))


MAE: 3295.5444178422294
MAPE: 0.27977775418798806
MSE: 29281568.984970607
RMSE: 5411.244679828349
r2_score train 0.8227704513775838
r2_score test 0.8139934799247438


In [ ]:
with open('../models/modelo_polinomial_regression.pkl', "wb") as archivo:
    pickle.dump(modelo_lr, archivo)

### Modelo DecisionTreeRegressor

In [ ]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor(max_depth=20, random_state=42)
tree_reg.fit(x_train, y_train)

predictions = tree_reg.predict(x_test)

print("MAE:", metrics.mean_absolute_error(y_test, predictions))
print("MAPE:", metrics.mean_absolute_percentage_error(y_test, predictions))
print("MSE:", metrics.mean_squared_error(y_test, predictions))
print("RMSE:", np.sqrt(metrics.mean_squared_error(y_test, predictions)))
print("r2_score train", lin_reg.score(x_train, y_train))
print("r2_score test",lin_reg.score(x_test, y_test))

MAE: 2966.160747500418
MAPE: 0.24944425430939107
MSE: 28731630.01717145
RMSE: 5360.189363928428
r2_score train 0.8227704513775838
r2_score test 0.8139934799247438


In [ ]:
with open('../models/modelo_DecisionTreeRegressor.pkl', "wb") as archivo:
    pickle.dump(modelo_lr, archivo)

### Pipeline (MODELO AUN POR ENTRENAR)

In [ ]:
reg_log = Pipeline(steps = [
    ("imputer", SimpleImputer()),
    ("scaler", StandardScaler()),
    ("reglog", LogisticRegression())
])
reg_log_param = {
    "imputer__strategy": ['mean', 'median'],
    "reglog__penalty": ['l1', 'l2'],
    "reglog__C": np.logspace(0, 4, 10)
}

rand_forest = RandomForestClassifier()
rand_forest_param = {
    "n_estimators": [10, 100, 1000],
    "max_features": [1,2,3]
}


svm = Pipeline(steps=[
    ("scaler", StandardScaler()),
    ("selectkbest", SelectKBest()),
    ("svm", SVC())
])


svm_param = {
    'selectkbest__k': [2, 3, 4],
    'svm__kernel': ['linear', 'rbf', 'sigmoid', 'poly'],
    'svm__C': [0.001, 0.1, 0.5, 1, 5, 10, 100],
    'svm__degree': [1,2,3,4],
    'svm__gamma': ['scale', 'auto']
}


gs_reg_log = GridSearchCV(reg_log,
                         reg_log_param,
                         cv = 10,
                         scoring = 'accuracy',
                         verbose = 1,
                         n_jobs = -1)

gs_rand_forest = GridSearchCV(rand_forest,
                         rand_forest_param,
                         cv = 10,
                         scoring = 'accuracy',
                         verbose = 1,
                         n_jobs = -1)

gs_svm = GridSearchCV(svm,
                         svm_param,
                         cv = 10,
                         scoring = 'accuracy',
                         verbose = 1,
                         n_jobs = -1)

grids = {"gs_reg_log": gs_reg_log,
        "gs_rand_forest": gs_rand_forest,
        "gs_svm": gs_svm}

In [ ]:
for nombre, grid_search in grids.items():
    grid_search.fit(x_train, y_train)